# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [ ]:
emergency_data = pd.read_csv(path+"응급실 정보.csv")
emergency_data.head()

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [ ]:
haversine

<function haversine.haversine.haversine(point1, point2, unit=<Unit.KILOMETERS: 'km'>, normalize=False, check=True)>

#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [ ]:
def find_nearest_emergency_rooms(emergency_data, input_coords, top_n=3, unit='km'):
    """
    입력된 좌표와 데이터프레임 내 응급실 좌표 간 거리를 계산하여 가장 가까운 응급실을 반환합니다.

    Parameters:
        emergency_data (DataFrame): 응급실 정보가 담긴 데이터프레임
        input_coords (tuple): 입력된 좌표 (위도, 경도)
        top_n (int): 반환할 응급실 개수
        unit (str): 거리 단위 ('km' 또는 'miles')

    Returns:
        DataFrame: 가장 가까운 응급실 정보와 거리
    """
    # 거리 계산
    emergency_data['거리'] = emergency_data.apply(
        lambda row: haversine(input_coords, (row['위도'], row['경도']), unit=unit),
        axis=1
    )

    # 거리 기준 정렬 및 상위 n개 반환
    nearest_emergency_rooms = emergency_data.sort_values(by='거리').head(top_n)
    return nearest_emergency_rooms

In [ ]:
input_coords = (37.5665, 126.9780)  # 서울의 위도, 경도
nearest_hospitals = find_nearest_emergency_rooms(emergency_data, input_coords, top_n=3, unit='km')

In [ ]:
nearest_hospitals

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,Distance,거리
23,강북삼성병원,서울특별시 종로구 새문안로 29 (평동),지역응급의료센터,02-2001-2001,02-2001-1000,37.568498,126.967938,0.914227,0.914227
180,서울적십자병원,"서울특별시 종로구 새문안로 9, 적십자병원 (평동)",지역응급의료기관,02-2002-8000,02-2002-8888,37.567155,126.966999,0.972375,0.972375
195,세란병원,서울특별시 종로구 통일로 256 (무악동),지역응급의료기관,02-737-0181,02-737-0181,37.575340,126.957707,2.040803,2.040803


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율
적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [ ]:
from math import radians, cos

def find_nearest_emergency_rooms_optimized(emergency_data, input_coords, top_n=3, initial_radius=10, unit='km'):
    """
    입력된 좌표를 기준으로 일정 반경 내의 응급실을 검색하며,
    추천 결과가 부족하면 반경을 확장하여 검색하는 함수.
    """
    radius = initial_radius
    R_km = 6378

    lat, lon = input_coords

    # 위도/경도 변화량을 계산 (1도 대략적인 거리)
    delta_lat = radius / R_km  # 위도 변화량
    delta_lon = radius / (R_km * cos(radians(lat)))  # 경도 변화량 (위도에 따라 조정)

    while True:
        # 해당 범위 내 응급실 필터링
        filtered_data = emergency_data[
            (emergency_data['위도'] >= lat - delta_lat) &
            (emergency_data['위도'] <= lat + delta_lat) &
            (emergency_data['경도'] >= lon - delta_lon) &
            (emergency_data['경도'] <= lon + delta_lon)
        ].copy()  # 명시적으로 복사

        # 필터링된 데이터가 있을 경우 거리 계산
        if not filtered_data.empty:
            filtered_data['Distance'] = filtered_data.apply(
                lambda row: haversine(input_coords, (row['위도'], row['경도']), unit=unit),
                axis=1
            )
            nearest_emergency_rooms = filtered_data.sort_values(by='Distance').head(top_n)

            # 조건을 만족하면 반환
            if len(nearest_emergency_rooms) >= top_n:
                return nearest_emergency_rooms

        # 조건을 만족하지 않으면 반경 확장
        radius += 10
        delta_lat = radius / R_km
        delta_lon = radius / (R_km * cos(radians(lat)))


In [ ]:
input_coords = (37.5665, 126.9780)  # 서울의 위도, 경도
nearest_hospitals_optimized = find_nearest_emergency_rooms_optimized(
    emergency_data, input_coords, top_n=5, initial_radius=1, unit='km'
)


In [ ]:
nearest_hospitals_optimized

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,Distance,거리
23,강북삼성병원,서울특별시 종로구 새문안로 29 (평동),지역응급의료센터,02-2001-2001,02-2001-1000,37.568498,126.967938,0.914227,0.914227
180,서울적십자병원,"서울특별시 종로구 새문안로 9, 적십자병원 (평동)",지역응급의료기관,02-2002-8000,02-2002-8888,37.567155,126.966999,0.972375,0.972375
195,세란병원,서울특별시 종로구 통일로 256 (무악동),지역응급의료기관,02-737-0181,02-737-0181,37.575340,126.957707,2.040803,2.040803
177,서울대학교치과병원,서울특별시 종로구 대학로 101 (연건동),응급실운영신고기관,1522-2700,02-6256-3362,37.580452,126.997196,2.295450,2.295450
176,서울대학교병원,서울특별시 종로구 대학로 101 (연건동),권역응급의료센터,1588-5700,02-2072-2475,37.579666,126.998963,2.357228,2.357228


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [ ]:
def load_api_keys(file_path):
    """
    JSON 파일에서 네이버 지도 API 클라이언트 ID와 키를 로드합니다.

    Parameters:
        file_path (str): API 키가 저장된 JSON 파일 경로

    Returns:
        dict: 클라이언트 ID와 키를 포함한 딕셔너리
    """
    try:
        with open(file_path, 'r') as file:
            keys = json.load(file)
        return keys
    except Exception as e:
        raise ValueError(f"API 키 파일 로드 중 오류 발생: {e}")

In [ ]:
keys = load_api_keys(path + 'map_key.txt')

c_id = keys.get('c_id')
c_key = keys.get('c_key')

print(f"c_id: {c_id}")
print(f"c_key: {c_key}")

#### 2) 함수 생성

In [ ]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    """
    Parameters:
        start_lat:
        start_lng:
        dest_lat:
        c_id:
        c_key:
    Returns:

    """
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        try:
            dist = data['route']['trafast'][0]['summary']['distance']
            dist_km = round(dist / 1000, 2) # km단위로 변경하기
            return dist_km
        except KeyError:
            raise ValueError("응답 데이터에서 거리 정보를 찾을 수 없습니다.")
    else:
        raise ConnectionError(f"API 호출 실패: {response.status_code}, {response.text}")

* 테스트

In [ ]:
start_lat, start_lng = 37.566535, 126.9779692 # 서울역 ? - 출처 : https://kr.toponavi.com/719
dest_lat, dest_lng = 35.1795543, 129.0756416  # 부산역 ? - 출처 : https://kr.toponavi.com/719

try:
    distance = get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key)
    print(distance)
except Exception:
    print(Exception)

408.39


In [ ]:
def find_nearest_emergency_rooms_with_api(emergency_data, input_coords, c_id, c_key, top_n=3, initial_radius=10):
    """
    네이버 지도 API를 사용하여 입력된 좌표를 기준으로 응급실과의 도로 기반 거리를 계산하며,
    추천 결과가 부족하면 반경을 확장하여 검색하는 함수.

    """
    radius = initial_radius  # 초기 반경
    lat, lon = input_coords

    while True:
        # 반경 내 대략적인 범위 필터링 (위도/경도를 이용한 초기 필터링)
        R_km = 6378
        delta_lat = radius / R_km
        delta_lon = radius / (R_km * cos(radians(lat)))

        filtered_data = emergency_data[
            (emergency_data['위도'] >= lat - delta_lat) &
            (emergency_data['위도'] <= lat + delta_lat) &
            (emergency_data['경도'] >= lon - delta_lon) &
            (emergency_data['경도'] <= lon + delta_lon)
        ].copy()

        if not filtered_data.empty:
            filtered_data['Distance'] = filtered_data.apply(
                lambda row: get_dist(
                    lat, lon, row['위도'], row['경도'], c_id, c_key
                ),
                axis=1
            )


            filtered_data = filtered_data[filtered_data['Distance'] < float('inf')]
            nearest_emergency_rooms = filtered_data.sort_values(by='Distance').head(top_n)

            if len(nearest_emergency_rooms) >= top_n:
                return nearest_emergency_rooms

        radius += 10

In [ ]:
input_coords = (37.5665, 126.9780)

find_nearest_emergency_rooms_with_api(emergency_data, input_coords, c_id, c_key, top_n=3, initial_radius=10)

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,Distance,거리
23,강북삼성병원,서울특별시 종로구 새문안로 29 (평동),지역응급의료센터,02-2001-2001,02-2001-1000,37.568498,126.967938,1.31,0.914227
180,서울적십자병원,"서울특별시 종로구 새문안로 9, 적십자병원 (평동)",지역응급의료기관,02-2002-8000,02-2002-8888,37.567155,126.966999,1.77,0.972375
177,서울대학교치과병원,서울특별시 종로구 대학로 101 (연건동),응급실운영신고기관,1522-2700,02-6256-3362,37.580452,126.997196,3.44,2.295450


#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

## **Mission Complete!**

수고 많았습니다!